L'objectif de ce projet est d'observer s'il existe des "déserts sportifs", 
lieux en France où les infrastrcutures sportives manquent. 
La question sera alors d'essayer d'expliquer ces déserts sportifs, que ce soit par des raisons économiques ou politiques. 
Enfin, il s'agira de comparer la carte des déserts sportifs avec d'autres cartes connues, à l'instar des déserts médicaux.

I/ Importation des modules

In [ ]:
#Installation des modules
!pip install py7zr geopandas openpyxl tqdm s3fs --quiet
!pip install PyYAML xlrd --quiet
!pip install cartiflette --quiet
!pip install contextily --quiet
!pip install folium --quiet
!pip install nb-clean --quiet
!pip install statsmodels --quiet

In [ ]:
#Importation des modules
import numpy as np
import numpy.linalg as al
import matplotlib.pyplot as plt 
import pandas as pd
import geopandas as gpd
from cartiflette import carti_download
import requests
import io
import zipfile
import folium
from folium.plugins import HeatMap
import statsmodels.formula.api as smf
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV


II/ Création de la base de données

In [ ]:
#Importation du jeu de données principal (localisation des équipements sportifs en France)

url = "https://data.sports.gouv.fr/api/explore/v2.1/catalog/datasets/equipements-sportifs/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
equipement = pd.read_csv(url, sep=";", low_memory=False)



#Importation d'un jeu de données csv sur des informations économiques et démographiques au niveau des communes

urlpop = "https://www.insee.fr/fr/statistiques/fichier/2521169/base_cc_comparateur_csv.zip"
response = requests.get(urlpop)

with zipfile.ZipFile(io.BytesIO(response.content)) as z:                 #io.BytesIO évite de télécharger le fichier
    with z.open("base_cc_comparateur.csv") as csvfile:                   #sur le disque dur
        df_communes = pd.read_csv(csvfile, sep=";", low_memory=False)


#Importation d'un jeu de données politiques au niveau des communes: résultats des législatives 2024 (2nd tour)

url2="https://www.data.gouv.fr/api/1/datasets/r/5a8088fd-8168-402a-9f40-c48daab88cd1"
legislatives2=pd.read_csv(url2, sep=";", low_memory=False)

In [ ]:
#Visualisation de la base de données principale
equipement.head(3)

In [ ]:
#Visualisation de la base de données sur les informations économiques et démographiques
df_communes.head(3)

In [ ]:
#Visualisation de la base de données politique
legislatives2.head(3)

In [ ]:
#On conserve les variables qui nous intéressent pour chaque jeu de données

#Pour le jeu de données principal
cols = ["Code Postal", "Commune nom", "Commune INSEE", "Département Code", "Densite Catégorie", "Type d'équipement sportif", 
        "Longitude", "Latitude"]
equipement = equipement[cols]

#Pour le jeu de données sur les informations économiques et démographiques
cols = ["CODGEO", "P22_POP", "P22_MEN","MED21", "TP6021", "P22_CHOM1564"]
df_communes = df_communes[cols]

#Pour le jeu de données politiques
cols = ["Code commune", "Libellé commune"]
cols += [(f"Nuance candidat {i}") for i in range(1, 5)]
cols += [(f"Elu {i}") for i in range(1, 5)]

legislatives2 = legislatives2[cols]

#Transformation de la variable élu en binaire pour tous les candidats
elu_cols = [f"Elu {i}" for i in range(1, 5)] 
for col in elu_cols:
    legislatives2[col] = legislatives2[col].notna().astype(int)  #Transforme les booléens en valeur 0 ou 1

In [ ]:
#Visualisation nouvelle base de données principale
equipement.head(3)

In [ ]:
#Visualisation nouvelle base de données sur les informations économiques et démographiques
df_communes.head(3)

In [ ]:
#Visualisation nouvelle base de données politique
legislatives2.head(3)

In [ ]:
#Dictionnaire des variables utilisées dans df_commune
pd.set_option("display.max_colwidth", None)

tab_communes = {
    "Variable": [
        "CODGEO", "P22_POP", "P22_MEN", "MED21", "TP6021", "P22_CHOM1564"
    ],
    "Description": [
        "Code INSEE de la commune",
        "Population en 2022",
        "Nombre de ménages en 2022",
        "Médiane du niveau de vie en 2021",
        "Taux de pauvreté en 2021",
        "Nombre de chômeurs de 15 à 64 ans en 2022"
    ]
}

dico_vars_communes = pd.DataFrame(tab_communes)
dico_vars_communes

In [ ]:
#Dictionnaire des variables utilisées dans legislatives


tab_pol = {
    "Variable": ["Code commune", "Libellé commune", "Nuance candidat", "Elu"],
    "Description": [
        "Code INSEE de la commune",
        "Nom de la commune",
        "Parti politique du candidat",
        "Variable binaire égale à 1 si le candidat a été élu, 0 sinon"
    ]
}

dico_vars_pol = pd.DataFrame(tab_pol)
dico_vars_pol

In [ ]:
#On souhaite créer une variable "Nuance politique du candidat élu" pour notre base de données finale

def get_nuance_elu(row):
    """
    Retourne la nuance du candidat élu pour la commune de la ligne "row"
    """
    if row['Elu 1']:
        return row['Nuance candidat 1']
    elif row['Elu 2']:
        return row['Nuance candidat 2']
    elif row['Elu 3']:
        return row['Nuance candidat 3']
    elif row['Elu 4']:
        return row['Nuance candidat 4']
    else:
        return np.nan

legislatives2['Nuance candidat élu'] = legislatives2.apply(get_nuance_elu, axis=1)

In [ ]:
legislatives2.head(3)

In [ ]:
#On garde seulement la nuance du candidat élu, ce qui nous intéresse ici
cols = ["Code commune", "Nuance candidat élu"]
legislatives2 = legislatives2[cols]
legislatives2.head(3)

In [ ]:
#On réunit tous les jeux de données pour obtenir notre jeu de données final

df_communes = df_communes.rename(columns={"CODGEO": "Commune INSEE"})
legislatives2 = legislatives2.rename(columns={"Code commune": "Commune INSEE"}) #On renomme les colonnes pour concaténer

df_final = (
    equipement
    .merge(df_communes, on="Commune INSEE", how="left")
    .merge(legislatives2, on="Commune INSEE", how="left")
)

df_final.head(12)

In [ ]:
#En faisant le test avec ma commune d'origine, on se rend compte que des lignes sont parfois en double, 
#voire triple, on va donc supprimer ces doublons.

test=df_final[df_final["Commune nom"] == "Eschau"]
test



In [ ]:
#Supression des lignes doublons
df_final = df_final.drop_duplicates()

In [ ]:
#Maintenant, on va regarder le type des colonnes pour mettre les variables dans le format
#qui nous arrange

print(df_final.dtypes)

In [ ]:
#On va convertir les Code de départements en entier

#On fait le choix de Remplacer 2A et 2B par 96 et 97
df_final['Département Code'] = df_final['Département Code'].replace({'2A': '96', '2B': '97'})

#On convertit en entier, coerce nous permet de gérer les erreurs en cas de valeur manquante
df_final['Département Code'] = pd.to_numeric(df_final['Département Code'], errors='coerce').astype('Int64')

In [ ]:
#Finalement, on ne conserve que les données sur la France métropolitaine, afin de faciliter les 
#représentations graphiques (cartes)

df_final = df_final[1 <= (df_final['Département Code'] <= 97)]

In [ ]:
df_final['MED21'] = pd.to_numeric(df_final['MED21'], errors='coerce').astype('Int64')

III/ Représentations graphiques des données

On va maintenant passer à une représentation graphique des données


In [ ]:
#On récupère le fond de carte de la france métropolitaine


#Téléchargement de toute la France avec les DROM
gdf = carti_download(
    values="France",
    crs=4326,
    borders="DEPARTEMENT",
    vectorfile_format="geojson",
    filter_by="FRANCE_ENTIERE",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

#Filtrer pour ne garder que la métropole (exclure les DROM)
departements_drom = ["971", "972", "973", "974", "975", "976"]
gdf = gdf[~gdf['INSEE_DEP'].isin(departements_drom)]

In [ ]:
gdf_pts = gpd.GeoDataFrame(
    df_final,
    geometry=gpd.points_from_xy(df_final["Longitude"], df_final["Latitude"]),
    crs="EPSG:4326"
)


fig, ax = plt.subplots(figsize=(20, 10))
gdf.boundary.plot(ax=ax, linewidth=0.4, color="gray")
gdf_pts.plot(ax=ax, markersize=3, alpha=0.7, color="red")
ax.set_xlim(-5.5, 10) 
ax.set_ylim(41, 51)
ax.set_title("Infrastructures sportives — France", fontsize=14)
ax.set_axis_off()
plt.savefig('docs/premiere_carte.png', dpi=300, bbox_inches='tight')

On voit que le schéma de la "diagonale du vide" semble se reproduire avec les infrastructures sportives.
La densité des infrastructures sportives en Corse semble moins importante qu'en France métropolitaine.

In [ ]:
#Carte de chaleur 


df = df_final.copy().dropna(subset=['Latitude', 'Longitude'])  #Supression des valeurs manquantes

#Création de la carte, centrée sur la France
calor = folium.Map(
    location=[46.5, 2.5],
    zoom_start=6,
    tiles='CartoDB positron'
)

#Récupération des données géographiques
heat_data = [[row['Latitude'], row['Longitude']] for idx, row in df.iterrows()]


#On complète la carte
HeatMap(
    heat_data,
    radius=15,           # Rayon de chaque point
    blur=20,             # Flou pour adoucir
    max_zoom=13,         # Zoom maximum
    gradient={            # Dégradé de couleurs personnalisé
        0.0: 'blue',
        0.3: 'lime',
        0.5: 'yellow',
        0.7: 'orange',
        1.0: 'red'
    }
).add_to(calor)
calor.save('docs/heatmap_toutes_infrastructures.html') #On sauvegarde la carte

In [ ]:
def carte_interactive(df, info_couleur, titre_legende, info_survol, chemin_sauvegarde):
    """

    Affiche un graphique et un tableau côte à côte à partir d'un DataFrame à 2 colonnes
    
    Paramètre
    ----------
    df : DataFrame
        DataFrame contenant les informations à représenter sur la carte interactive
        
    info_couleur : str
        Nom de la variable du DataFrame selon laquelle on veut colorer la carte

    titre_legende : str
        Titre de la légende de couleurs

    info_survol : list
        Liste des variables (str) pour lesquelles on veut afficher les informations au survol de la souros

    chemin_sauvegarde : str
        Endroit où l'on veut sauvegarder la carte
    """

    #On copie le fond de carte pour ne pas le modifier directement
    gd = gdf.copy()
    
    
    #Fusion des données selon les codes de département
    gd = gd.merge(                 
    infra_par_dept, 
    left_on='INSEE_DEP', 
    right_on='Département Code', 
    how='left'
    )

    #Création de la carte, centrée sur la France
    m = folium.Map(
        location=[46.5, 2.5],
        zoom_start=6,
        tiles='CartoDB positron'
    )

    #On complète la carte
    folium.Choropleth(
        geo_data=gd,
        name='choropleth',
        data=df,
        columns=['Département Code', info_couleur],
        key_on='feature.properties.INSEE_DEP',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=titre_legende
    ).add_to(m)


    #Ajout des informations au survol de la souris
    folium.GeoJson(
        gd,
        style_function=lambda x: {'fillColor': 'transparent', 'color': 'transparent'},
        tooltip=folium.GeoJsonTooltip(
            fields=['INSEE_DEP'] + info_survol,
            localize=True
        )
    ).add_to(m)
    m.save(chemin_sauvegarde)

In [ ]:
def format_dept_code(code):
    """
    Ajouter un zéro devant les codes à 1 chiffre (1 → 01)
    """
    if code in ['2A', '2B']:
        return code
    if int(code) >= 10:
        return code
    else:
        return "0" + code

In [ ]:
#On crée une carte qui compte le nombre d'infrastructures sportives par département

infra_par_dept = df_final.copy().groupby('Département Code').size().reset_index(name="Nombre d'infrastructures")

#On réadapte les codes de département pour correspondre au fond de carte
infra_par_dept['Département Code'] = infra_par_dept['Département Code'].astype(str)
infra_par_dept['Département Code'] = infra_par_dept['Département Code'].replace({
    '96': '2A',
    '97': '2B'
})


infra_par_dept['Département Code'] = infra_par_dept['Département Code'].apply(format_dept_code)
print("-" * 50)
print("Visualisation de la base de données infra_par_dept")
print("-" * 50)
print(infra_par_dept.head(4))

carte_interactive(infra_par_dept, "Nombre d'infrastructures", "Nombres d'infrastructures sportives",
                  ["Nombre d'infrastructures"], 'docs/carte_departements.html')

On observe sur cette carte que la Moselle et le Nord sont les départements comptant le plus grand nombre d’infrastructures sportives, tandis que les deux départements corses en disposent nettement moins.
Cependant, cette lecture doit être nuancée : la carte ne tient pas compte du nombre d’habitants ni du revenu médian de chaque département, deux facteurs essentiels pour interpréter correctement ces écarts.
Observons maintenant l'effet de la population sur cette carte.

In [ ]:
#Estimation du nombre d'habitants par département avec les communes disponibles dans la base de données
pop_par_dept = df_final.copy()[['Département Code', 'Commune INSEE', 'P22_POP']].drop_duplicates(
    subset=['Commune INSEE']
).groupby('Département Code')['P22_POP'].sum().reset_index()


pop_par_dept['Département Code'] = pop_par_dept['Département Code'].astype(str)
pop_par_dept['Département Code'] = pop_par_dept['Département Code'].replace({
    '96': '2A',
    '97': '2B'
})

pop_par_dept.columns = ['Département Code', 'Population totale']
pop_par_dept['Département Code'] = pop_par_dept['Département Code'].apply(format_dept_code)

infra_par_dept = infra_par_dept.merge(pop_par_dept, on='Département Code', how='left')

infra_par_dept['Infrastructures pour 10000 habitants'] = (
    infra_par_dept["Nombre d'infrastructures"] / infra_par_dept['Population totale'] * 10000
).round(2) #Calcul du nombre d'infrastructures pour 10000 habitants

print("-" * 50)
print("Visualisation de la base de données infra_par_dept")
print("-" * 50)
print(infra_par_dept.head(4))


carte_interactive(infra_par_dept, 'Infrastructures pour 10000 habitants', 'Infrastructures sportives pour 10000 habitants', 
                  ["Nombre d'infrastructures", "Population totale", "Infrastructures pour 10000 habitants"],
                  'docs/carte_infra_par_habitant.html')

Le résultat change totalement puisque c'est désormais les Hautes Alpes qui détiennent le plus d'infrastructures sportives par 10 000 habitants.

On va maintenant passer à des statistiques descriptives numériques 

In [ ]:
print(f"Il y a {df_final["Type d'équipement sportif"].count()} infrastructures sportives dans notre base de données")

In [ ]:
print("Voici le type d'infrastructures qu'on retrouve dans notre base de données : \n \n")
df_final.copy()["Type d'équipement sportif"].value_counts().reset_index().head(15)


In [ ]:
infra_par_commune = (
    df_final.copy().groupby("Commune INSEE")
            .size()
            .reset_index(name="Nombre d'infrastructures par commune")
)

infra_par_commune.describe()

In [ ]:
communes_ref = df_final.copy()[['Commune INSEE', 'Commune nom']].drop_duplicates()
infra_par_commune = infra_par_commune.merge(
    communes_ref,
    on="Commune INSEE",
    how="left"
)
infra_par_commune.sort_values("Nombre d'infrastructures par commune", ascending=False).head(10)

In [ ]:
arrondissements_paris = df_final.copy()[
    df_final["Commune INSEE"].astype(str).between("75100", "75120")
]

infra_par_arrdt_Paris = (
    arrondissements_paris
    .groupby("Commune nom")
    .size()
    .reset_index(name="Nombre d'infrastructures")
    .sort_values("Nombre d'infrastructures", ascending=False)
)

arrondissements_marseille = df_final.copy()[
    df_final["Commune INSEE"].astype(str).between("13201", "13216")
]

infra_par_arrdt_Marseille = (
    arrondissements_marseille
    .groupby("Commune nom")
    .size()
    .reset_index(name="Nombre d'infrastructures")
    .sort_values("Nombre d'infrastructures", ascending=False)
)

# Arrondissements de Lyon : 69381 à 69389
arrondissements_lyon = df_final.copy()[
    df_final["Commune INSEE"].astype(str).between("69381", "69389")
]

infra_par_arrdt_Lyon = (
    arrondissements_lyon
    .groupby("Commune nom")
    .size()
    .reset_index(name="Nombre d'infrastructures")
    .sort_values("Nombre d'infrastructures", ascending=False)
)

total_paris = infra_par_arrdt_Paris["Nombre d'infrastructures"].sum()
total_lyon = infra_par_arrdt_Lyon["Nombre d'infrastructures"].sum()
total_marseille = infra_par_arrdt_Marseille["Nombre d'infrastructures"].sum()

print("Total Paris :", total_paris)
print("Total Lyon :", total_lyon)
print("Total Marseille :", total_marseille)

villes_majeures = pd.DataFrame([
    {"Commune INSEE": "75056", "Nombre d'infrastructures par commune": total_paris,      "Commune nom": "Paris"},
    {"Commune INSEE": "69123", "Nombre d'infrastructures par commune": total_lyon,       "Commune nom": "Lyon"},
    {"Commune INSEE": "13055", "Nombre d'infrastructures par commune": total_marseille,  "Commune nom": "Marseille"},
])
infra_par_commune = pd.concat([infra_par_commune, villes_majeures], ignore_index=True)

infra_par_commune.sort_values("Nombre d'infrastructures par commune", ascending=False).head(10)




Le résultat donnée nous semble un peu plus logique car ce sont les 3 villes les plus peuplées de France et il semblait bizarre de ne pas les voir dans le classement précédent. Paris est donc de loin la ville avec le plus d'infrastructures. Un question légitime (en connaissant la ville) est de se demander lesquelles sont-elles?

In [ ]:
df_paris = df_final.copy()
df_paris = df_paris[
    df_paris["Commune INSEE"].astype(str).between("75101", "75120")
]

top_equipements_paris = (
    df_paris["Type d'équipement sportif"]
    .value_counts()
    .reset_index()
    .rename(columns={"count": "Nombre"})
    .head(10)
)

top_equipements_paris

In [ ]:
infra_par_dept.sort_values("Nombre d'infrastructures", ascending=False)

In [ ]:
infra_par_dept.sort_values("Infrastructures pour 10000 habitants", ascending=False)

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(infra_par_commune["Nombre d'infrastructures par commune"], bins=100)
plt.yscale("log")
plt.xlabel("Nombre d'infrastructures par commune")
plt.ylabel("Nombre de communes (échelle log)")
plt.title("Distribution (échelle logarithmique)")
plt.show()




Le graphique obtenu ressemble la densité d'une loi exponentielle: Toutes les valeurs sont positives mais la décroissance est très forte avec juste quelques extrêmes.

In [ ]:
infra_par_densite = (
    df_final.copy().groupby("Densite Catégorie")
            .size()
            .reset_index(name="Nombre d'infrastructures")
)
infra_par_densite = infra_par_densite.sort_values("Nombre d'infrastructures", ascending=False)
infra_par_densite

Selon la classification de l’INSEE, les communes rurales concentrent le plus grand nombre d’infrastructures sportives en valeur absolue, notamment les catégories « Rural à habitat dispersé » et « Bourgs ruraux ». Ce résultat s’explique principalement par le fait que les communes rurales représentent la grande majorité des communes françaises. Les grands centres urbains totalisent également beaucoup d’équipements, mais restent derrière car ils sont moins nombreux. Ainsi, cette répartition reflète surtout le nombre de communes par catégorie, davantage que leur niveau réel d’équipement moyen.

In [ ]:
def tab_barplot(df, titre):
    """
    Affiche un graphique et un tableau côte à côte à partir d'un DataFrame à 2 colonnes
    
    Paramètre
    ----------
    df : DataFrame
        DataFrame à deux colonnes
        
    titre : str
        Titre de la figure
    """
    fig = plt.figure(figsize = (16, 6))
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)


    #Création du barplot
    ax1.bar(
    df[df.columns[0]], 
    df[df.columns[1]]
    )
    ax1.set_xlabel(df.columns[0])
    ax1.set_ylabel(df.columns[1])



    #Création du tableau
    ax2.axis('off')

    table = ax2.table(
        cellText=df.values,
        colLabels=df.columns,
        cellLoc='center',
        loc='center',
        colWidths=[0.4, 0.6]
    )

    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2) 

    fig.suptitle(titre)

In [ ]:
df_med = infra_par_commune.merge(
    df_final.copy()[["Commune nom", "MED21"]],
    on= "Commune nom",
    how="left"
)


df_med["Quartile Revenu"] = pd.qcut(
    df_med["MED21"],
    q=4,
    labels=["Q1", "Q2", "Q3", "Q4"]
)

infra_par_quartile = (
    df_med.groupby("Quartile Revenu", observed = True)["Nombre d'infrastructures par commune"]
            .mean()
            .reset_index()
)

infra_par_quartile.columns = ["Quartile Revenu", "Nombre d'infrastructures moyen par commune"]

tab_barplot(infra_par_quartile, "Nombre d'infrastructures moyen par commune par quartile de revenu médian")

D’après la distribution en quartiles du revenu médian des communes, on observe une relation contrastée entre niveau de vie et nombre d’infrastructures sportives. Les communes situées dans les quartiles Q1 et Q2 - les territoires les plus modestes - disposent en moyenne du plus grand nombre d’équipements, autour de 80 à 90 infrastructures par commune. À l’inverse, les communes appartenant aux quartiles Q3 et surtout Q4 (les plus aisées) présentent des moyennes nettement plus faibles, autour de 40 infrastructures. Cette répartition suggère que les territoires moins favorisés accueillent proportionnellement davantage d’infrastructures sportives que les plus riches, sans doute en raison de leur poids démographique ou de logiques d’aménagement spécifiques.

In [ ]:
df_chom = infra_par_commune.merge(
    df_final.copy()[["Commune nom", "P22_CHOM1564"]],
    on= "Commune nom",
    how="left"
)


df_chom["Quartile Chômage"] = pd.qcut(
    df_chom["P22_CHOM1564"],
    q=4,
    labels=["Q1", "Q2", "Q3", "Q4"]
)

infra_par_quartile_chom = (
    df_chom.groupby("Quartile Chômage", observed = True)["Nombre d'infrastructures par commune"]
            .mean()
            .reset_index()
)

infra_par_quartile_chom.columns = ["Quartile Chômage", "Nombre d'infrastructures moyen par commune"]
tab_barplot(infra_par_quartile_chom, "Nombre d'infrastructures moyen par commune par quartile de chômage")

L’analyse par quartiles du taux de chômage montre une relation positive à celle observée avec le revenu médian. Ici, les communes les plus touchées par le chômage (quartile Q4) concentrent en moyenne beaucoup plus d’infrastructures sportives, près de 170 par commune, contre seulement 9 à 15 dans les quartiles les plus faibles (Q1–Q2). L’étude du revenu médian révélait de la même manière que les communes les plus aisées étaient plutôt moins dotées en infrastructures. Ces résultats suggèrent qu’une partie importante de l’offre sportive se situe dans des territoires plus fragiles socio-économiquement, où le niveau de chômage est élevé et les revenus plus faibles.

In [ ]:
df_pol = infra_par_commune.merge(
    df_final.copy()[["Commune INSEE", "Nuance candidat élu"]],
    on= "Commune INSEE",
    how="left"
)


infra_par_pol= (
    df_pol.groupby("Nuance candidat élu", observed = True)["Nombre d'infrastructures par commune"]
            .mean()
            .reset_index()
)

infra_par_pol.columns = ["Nuance du candidat élu", "Nombre d'infrastructures moyen par commune"]
infra_par_pol = infra_par_pol.sort_values("Nombre d'infrastructures moyen par commune", ascending=False)

tab_barplot(infra_par_pol, "Nombre d'infrastructures moyen par commune selon la nuance politique du candidat élu")

La comparaison du nombre moyen d’infrastructures sportives selon la nuance du candidat élu en 2024 montre des écarts importants entre territoires. Les communes ayant élu un candidat classé DIV, UG ou HOR apparaissent en moyenne nettement plus dotées en équipements que celles administrées par des élus LR, REG, DVD ou ECO. Toutefois, cette lecture doit être fortement nuancée : l’élection de 2024 n’a eu quasiment aucun impact sur la présence d’infrastructures sportives, puisque celles-ci résultent d’investissements réalisés sur de longues périodes. Les différences observées reflètent donc avant tout les caractéristiques socio-économiques des territoires où ces nuances politiques sont dominantes, plutôt qu’un effet direct de l’orientation politique des élus élus en 2024.

IV/ Modélisation

On va maintenant faire la partie modélisation.

In [ ]:
#On transforme notre base de données pour faire des régressions

infra_count = df_final.groupby('Commune INSEE').size().reset_index(name='Nb_infrastructures')

autres_vars = df_final.groupby('Commune INSEE').first().reset_index()

df_commune = autres_vars.merge(infra_count, on='Commune INSEE', how='left')

df_commune.head(6)


In [ ]:
#On renomme les variables pour ne pas avoir d'erreurs avec statsmodels (accents et espaces)
df_commune = df_commune.rename(columns={
    "Nuance candidat élu": "Nuance_candidat",
    "Densite Catégorie": "Densite"
})

In [ ]:
#Determinants du nombre d'infrastructures
model = smf.ols('Nb_infrastructures ~ P22_POP + MED21 + C(Nuance_candidat) + C(Densite)', data = df_commune.dropna()).fit(cov_type='HC3') #Robuste à l'hétéroscédasticité


print(model.summary())

L’analyse montre que la couleur politique du député élu en 2024 n’a pas d’impact significatif sur le nombre d’infrastructures sportives, ce qui s’explique par des décisions d’aménagement prises sur le long terme.​
En revanche, la densité du territoire apparaît comme un facteur déterminant dans plusieurs types d’espaces (urbains et ruraux), influençant la répartition des équipements.​
La population en 2022 joue également un rôle majeur : plus un territoire compte d’habitants, plus il tend à disposer d’un parc sportif développé.​
Le revenu médian de 2021 intervient lui aussi, car d'après la régression, une hausse de 1€ du revenu médian diminue le nombre d'infrastructures d'une commune de 0,0004. 
Concentrons nous sur ces 2 dernières variables:

In [ ]:
model2 = smf.ols('Nb_infrastructures ~ P22_POP + MED21', data = df_commune.dropna()).fit(cov_type='HC3') #Robuste à l'hétéroscédasticité


print(model2.summary())


On constate qu'à eux seuls, le revenu médian de 2021 et la population en 2022 expliquent 78% des infrastructures sportives en France. 

Si on se concentre uniquement sur la population:

In [ ]:
model3 = smf.ols('Nb_infrastructures ~ P22_POP', data = df_commune).fit(cov_type='HC3') #Robuste à l'hétéroscédasticité


print(model3.summary())

In [ ]:
fig = plt.figure(figsize = (10, 6))
ax = fig.add_subplot(1, 1, 1)

# Nuage de points
ax.scatter(df_commune['P22_POP'], df_commune['Nb_infrastructures'], 
           alpha=0.5, s=30, label='Observations')
ax.plot(df_commune['P22_POP'].dropna(), model3.predict(), color = "Red", label = "Droite de régression")


Cela nous incite à créer une nouvelle variable dans notre base de données : nombre d'infrastructures pour 10 000 habitants

In [ ]:
df_commune["infra_10k"] = (df_commune["Nb_infrastructures"] / df_commune["P22_POP"]) * 10000
df_commune.head(6)

In [ ]:
model = smf.ols('infra_10k ~ MED21 + C(Nuance_candidat) + C(Densite)', data = df_commune.dropna()).fit(cov_type='HC3') #Robuste à l'hétéroscédasticité


print(model.summary())

Sans prendre en compte la population en 2022, le R^2 chute et ne vaut seulement 0,08. 

In [ ]:
df_cluster = df_commune.copy()
df_cluster = df_cluster.dropna()

df_cluster = pd.get_dummies(df_cluster, 
                            columns=["Densite"], 
                            drop_first=False)

vars_clustering = [col for col in df_cluster.columns 
                   if col.startswith("Densite_")] + [
    "Nb_infrastructures", "MED21", "P22_POP"
]

X = df_cluster[vars_clustering]


# standardisation (important)
X_scaled = StandardScaler().fit_transform(X)


# clustering en 4 groupes
kmeans = KMeans(n_clusters=4, random_state=42)
df_cluster["cluster"] = kmeans.fit_predict(X_scaled)

df_cluster.head(6)

In [ ]:
# Préparation des données
df_lasso = df_commune.copy()

# Variables numériques
vars_numeriques = ['P22_POP', 'MED21', 'P22_CHOM1564']

# Variables catégorielles
vars_categorielles = ['Nuance_candidat', 'Densite']

# Sélection des variables
colonnes = vars_numeriques + vars_categorielles + ['Nb_infrastructures']
df_lasso = df_lasso[colonnes].dropna()

#Création des variables binaires (drop_first=True évite la colinéarité)
df_avec_dummies = pd.get_dummies(
    df_lasso,
    columns=vars_categorielles,  
    drop_first=True,
    prefix=['Nuance', 'Den']
)

#Séparation pour standardiser X
y = df_avec_dummies['Nb_infrastructures']
X = df_avec_dummies.drop('Nb_infrastructures', axis=1)

#Standardisation X
X_scaled = StandardScaler().fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

lasso_cv = LassoCV(
    cv=10,              # 10-fold cross-validation
    random_state=42,
    max_iter=10000,
    n_alphas=100        # Teste 100 valeurs d'alpha différentes
)

# Entraînement
lasso_cv.fit(X_scaled, y)

coefficients = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': lasso_cv.coef_,
    'Coefficient_abs': np.abs(lasso_cv.coef_)
}).sort_values('Coefficient_abs', ascending=False)

print("Tous les coefficients (triés par importance) :")
coefficients


In [ ]:
#FAIRE UNE REGRESSION LOGISTIQUE AVEC LA PROBABILITE DETRE UN DESERT SPORTIF (TROUVER UN SEUIL ou <= Xinfra_10k = desert_sportif)

In [ ]:
df_final_cluster = df_final.merge(
    df_cluster[["Commune INSEE", "cluster"]],
    on="Commune INSEE",
    how="left"
)

gdf_pts = gpd.GeoDataFrame(
    df_final_cluster,
    geometry=gpd.points_from_xy(df_final_cluster["Longitude"], df_final_cluster["Latitude"]),
    crs="EPSG:4326"
)


fig, ax = plt.subplots(figsize=(20, 10))
gdf.boundary.plot(ax=ax, linewidth=0.4, color="gray")
gdf_pts.plot(ax=ax, markersize=3, alpha=0.7, column="cluster", cmap = "tab10", legend = True)
ax.set_xlim(-5.5, 10) 
ax.set_ylim(41, 51)
ax.set_title("Infrastructures sportives colorées selon les cluster", fontsize=14)
ax.set_axis_off()
plt.savefig('docs/carte_clusters.png', dpi=300, bbox_inches='tight')

In [ ]:
#Je sais pas pourquoi il manque des points (peut être à cause de dropna dans le clustering?)
#Essayer de commenter quand même 

In [64]:
!nb-clean clean Projet.ipynb
!ls -lh Projet.ipynb